# Decoding

Decoding the outputs from language models is an often overlooked part of language models. Typically, we focus on the model which is important, but does not represent the whole solution.

We can think of it like a car. The model we build is the engine and inner mechanics of the car - they're pretty important. However, decoding is the wheels - the one-and-only connection between the engine and the road. If you're using the wheels are dysfunctional, it doesn't matter whether you have the engine of a Ferrari or a Fiat Punto, performance will suffer.

So, with that I hope it's clear that taking some time to understand what decoding is, and which methods are available, is important and worth some time.

![Model with decoding section](../../assets/images/random_sampling.png)

*Decoding operations all take place at the end of a language model.*

The decoding operation takes place at the output of our language model. Typically we would be looking at a probability distribution across our *output vocabulary* tensor. Which means a set of values between 0 and 1 where each value maps to a specific token in our vocabulary - where tokens are usually words, characters, or *SentencePieces*.

The default behavior we can expect when looking at decoding is to take the token with the highest probability - this is called *greedy decoding*. We will cover greedy decoding and why this may not always be the best option very soon. Alongside greedy decoding, we have many other alternative approaches for decoding, such as *random sampling*, *beam search*, and *minimum Bayes risk (MBR)*.

In this chapter, we will be initializing a GPT-2 transformer model for text generation using the code in **fig.TK**

In [10]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

sequence = ("He began his premiership by forming a five-man war cabinet which included"
            "Chamerlain as Lord President of the Council, Labour leader Clement Attlee as"
            "Lord Privy Seal (later as Deputy Prime Minister), Halifax as Foreign Secretary"
            "and Labour's Arthur Greenwood as a minister without portfolio. In practice,")

inputs = tokenizer.encode(sequence, return_tensors='pt')

## Greedy Decoding

Greedy decoding is the simplest method and likely the natural, more intuitive approach for most of us. With this approach we take the most probable token output by our model (argmax) - no questions asked.

<span style='color:red'>Add math here</span>

Taking this approach works well for shorter sequences, but when applying this method to longer sequences we will find that our model begins repeating itself. The reason for this is that the model will be inclined to predict assign a specific token a high probability if it follows a token that it is often paired with in language. We find this repetition will often be initiated after a punctuation mark such as a full-stop. After our model sees the full-stop, it will consider all of the previous instances of a full-stop that it has previously seen, and likely find that the token *'The'* is most probable to follow a full-stop.

So, when we apply this decoding approach to a longer sequence of text, the model is likely to fall into this trap. As soon as the model has predicted a few repeating words it will continue doing so as it becomes stuck in this greedy decoding induced highest-probability matching loop. We can see this happening with our Churchill extract in **fig.TK**.

```diff
He began his premiership by forming a five-man war cabinet which included
Chamerlain as Lord President of the Council, Labour leader Clement Attlee as
Lord Privy Seal (later as Deputy Prime Minister), Halifax as Foreign Secretary
and Labour's Arthur Greenwood as a minister without portfolio. In practice,
+ the cabinet was divided into three parts: the Cabinet of Ministers, the
+ Cabinet of Ministers of the Crown, and the Cabinet of Ministers of the Crown.
+ The Cabinet of Ministers was the most important part of the government. The 
+ Cabinet of Ministers was the most important part of the government. The 
+ Cabinet of Ministers was the most important part of the government. The 
+ Cabinet of Ministers was the most important part of the government. The 
+ Cabinet of Ministers was the most important part of the government. The 
+ Cabinet of Ministers was the most important part of the government. The 
+ Cabinet of Ministers was the most important part of the government. The 
+ Cabinet of Ministers was the most important part of the government. The 
+ Cabinet of Ministers...
```

Greedy decoding is the default method when generating text with the transformers GPT-2 implementation. To build this out in code, all we need to write is the following:

In [11]:
outputs = model.generate(inputs, max_length=200)

tokenizer.decode(outputs[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"He began his premiership by forming a five-man war cabinet which includedChamerlain as Lord President of the Council, Labour leader Clement Attlee asLord Privy Seal (later as Deputy Prime Minister), Halifax as Foreign Secretaryand Labour's Arthur Greenwood as a minister without portfolio. In practice, he was a member of the cabinet of the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democrats, the Liberal Democr

We can see in this example that our code has gotten caught on `, the Liberal Democrats` which repeats again and again. It's also worth noting that the line `tokenizer.decode(outputs[0])` refers to decoding our token IDs into human readable text, **not** greedy decoding (or any of the other decoding methods). Our decoding method is configured and implemented through the `model.generate()` method.

## Random Sampling

Our next option is **random sampling**. Much like before we have our model which outputs a probability distribution across our output vocabulary tensor. Unlike before, rather than selecting the token with the highest probability, we randomly choose a token based on the probability distribution.

What that means is if we had the following words and probabilities:

```python
'he': 0.01,
'great': 0.21,
'influence': 0.48,
    ...
'deal': 0.02
```

Our random sampling approach would have a *1%* chance of choosing `'he'`, a *21%* chance of choosing `'great'`, a *48%* chance of `'influence'`, and a *2%* chance of `'deal'`. This method is most likely to sampling `'influence'`, but it is not guarenteed 100% of the time as it was with greedy decoding - and that is enough for us to avoid falling into a repetative loop.

To implement this in the transformers library, we add `do_sample=True` to `generate`:

In [12]:
outputs = model.generate(inputs, max_length=200, do_sample=True)

tokenizer.decode(outputs[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"He began his premiership by forming a five-man war cabinet which includedChamerlain as Lord President of the Council, Labour leader Clement Attlee asLord Privy Seal (later as Deputy Prime Minister), Halifax as Foreign Secretaryand Labour's Arthur Greenwood as a minister without portfolio. In practice, these positions were held by men who did not know a thing or an entire people.\n\nOn that occasion, I was given a job, though not by an ambassador, and I was called to the office of the Crown Commissioner of London before I was called in to work. I was given the job and I accepted, in principle, the role of the commissioner. I was at no point required to make an appointment, which would have left me with a four-week work week. On no other occasion should I have taken the role of the commissioner for which I deserved a leave of absence, since, as always, I was the only person who had to come back and see a different set"

We can see immediately that repetition is no longer a problem. However, we now have a new problem, we will often find that this approach can be too random and lacks coherence.

So on one side, we have greedy search which is **too strict** for generating text - on the other we have random sampling which produces wonderful **gibberish**. We need to find something that does a little of both.

## Beam Search

Whereas greedy decoding and random sampling calculate the best option based on the very next word/token only - beam search checks for multiple word/tokens into the future and assesses the quality of all of these tokens combined.

![Beam search](../../assets/images/beam_search.png)

From this search, we will return multiple potential output sequences - the number of options we consider is the number of 'beams' we search with. However, because we are now back to ranking sequences and selecting the most probable - beam search can cause our text generation to again degrade into repetitive sequences:

In [13]:
outputs = model.generate(inputs, max_length=200, do_sample=True, num_beams=5)

tokenizer.decode(outputs[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"He began his premiership by forming a five-man war cabinet which includedChamerlain as Lord President of the Council, Labour leader Clement Attlee asLord Privy Seal (later as Deputy Prime Minister), Halifax as Foreign Secretaryand Labour's Arthur Greenwood as a minister without portfolio. In practice, the cabinet consisted of three ministers, one of whom was appointed as a member of the Cabinet of Ministers and the other two as a member of the Cabinet of Ministers.\n\nChamerlain was a member of the House of Lords and a member of the Cabinet of Ministers, but was not a member of the Cabinet of Ministers.\n\nChamerlain was also a member of the Cabinet of Ministers but was not a member of the Cabinet of Ministers.\n\nChamerlain was a member of the Cabinet of Ministers but was not a member of the Cabinet of Ministers.\n\nChamerlain was a member of the Cabinet of Ministers but was not a member of the Cabinet of Ministers.\n"

To counteract this, we increase the decoding temperature - which controls the amount of randomness in the outputs. The default temperature is `1.0` - pushing this to a higher value of `1.5` makes a huge difference:

In [15]:
outputs = model.generate(inputs, max_length=200, do_sample=True, num_beams=5, temperature=1.5)

tokenizer.decode(outputs[0])

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


"He began his premiership by forming a five-man war cabinet which includedChamerlain as Lord President of the Council, Labour leader Clement Attlee asLord Privy Seal (later as Deputy Prime Minister), Halifax as Foreign Secretaryand Labour's Arthur Greenwood as a minister without portfolio. In practice, though, it was hard to find anyone in the Cabinet who could play any role. Chamerlain made his first cabinet appearance on 19 July 1940, when he joined the Cabinet as Minister for Foreign Affairs. When he took over as Minister of State on 6 September 1940, he became the most senior cabinet minister ever to be appointed by a member of the Cabinet, becoming Lord President of the Council, Labour leader and then as Lord President of the Lords.Chamerlain, however, did not play any significant role as Lord President until May 1940, when he took over as Minister of Public Works. In the summer of 1940, Chamerlain was appointed Minister of State for the Crown by the High Court"